In [11]:
import ee

In [5]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWh-xRP6hbd2IxTmOLdYkE3tQPdz5xCScekB4wW2KUAs6sk2vuLRlns

Successfully saved authorization token.


In [6]:
ee.Initialize()

In [8]:
prudhoebay = ee.FeatureCollection("users/bibekaryal7/ABoVE/PrudhoeBay")
brw_atq_ivotuk = ee.FeatureCollection("users/bibekaryal7/ABoVE/Brw_Atq_Ivotuk")
anwr = ee.FeatureCollection("users/bibekaryal7/ABoVE/ANWR_NAD83_Z6")

In [10]:
regionList = [prudhoebay, brw_atq_ivotuk, anwr]
nameList = ['prudhoebay', 'brw_atq_ivotuk', 'anwr']

In [47]:
modisNDVIlist = ee.ImageCollection('MODIS/006/MOD13A1').filter(ee.Filter.date('2000-02-18', '2021-07-29')).select("NDVI")

In [51]:
n_images = modisNDVIlist.size().getInfo()
modisNDVIlist = modisNDVIlist.toList(n_images)

In [64]:
for i in range(n_images): #Iterate through all images
  img = ee.Image(modisNDVIlist.get(i)) #get current image
  for j in range(len(regionList)): #Iterate through all regions
    clipped_image = img.clip(regionList[j]); #Clip image by region to reduce storage space
    img_name = clipped_image.get('system:index');
    img_name = img_name.getInfo();
    task = ee.batch.Export.image.toDrive(
        image=clipped_image,
        region=regionList[j].geometry(),
        description=img_name,
        folder=nameList[j],
        scale=30,
        maxPixels=3101797300
    )
    task.start()
  break